# Imports

In [ ]:
# Standard Libraries
import os
import io
import math
import time
import random
import tempfile
import warnings
from multiprocessing import Manager
from typing import Optional

# Third-party Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import SimpleITK as sitk
import torch
import torch.nn as nn
import torchio as tio
from torch.utils.data import DataLoader, Dataset, random_split
from torch.cuda.amp import GradScaler, autocast

# MONAI Libraries
from monai.config import print_config
from monai.data import DataLoader
from monai.utils import set_determinism

# Custom Libraries
from generative.inferers import DiffusionInferer
from generative.networks.nets import DiffusionModelUNet
from generative.networks.schedulers import DDPMScheduler, DDIMScheduler
from dataloader import Train ,Eval
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
# Configuration
sitk.ProcessObject.SetGlobalDefaultThreader("Platform")
warnings.filterwarnings('ignore')


JUPYTER_ALLOW_INSECURE_WRITES=True

In [ ]:
# !pip install statsmodels

In [ ]:
import statsmodels.stats.multitest as mt

# Set seeds and configs

In [ ]:
# Initialize Configuration
config = {
    'batch_size': 1,
    'imgDimResize': (160, 192, 160),
    'imgDimPad': (208, 256, 208),
    'spatialDims': '3D',
    'unisotropic_sampling': True,
    'perc_low': 1,
    'perc_high': 99,
    'rescaleFactor': 2,
    'base_path': '',
}

# Seed and Device Configuration
manualSeed = 999
random.seed(manualSeed)
torch.manual_seed(manualSeed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CUDA and CUDNN Configuration
# Uncomment the following line to specify CUDA_VISIBLE_DEVICES
# os.environ['CUDA_VISIBLE_DEVICES'] = '2,3,5,6'
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# SimpleITK Configuration
# Set the default number of threads and global behavior for SimpleITK
sitk.ProcessObject.SetGlobalDefaultThreader("Platform")



# Load the data

In [ ]:

imgpath = {}
# '/acmenas/hakrami/patched-Diffusion-Models-UAD/Data/splits/BioBank_train.csv'
#'/acmenas/hakrami/patched-Diffusion-Models-UAD/Data/splits/IXI_train_fold0.csv',
#csvpath_trains = ['/project/ajoshi_27/akrami/patched-Diffusion-Models-UAD/Data/splits/BioBank_train.csv', '/project/ajoshi_27/akrami/patched-Diffusion-Models-UAD/Data/splits/BioBank_train.csv']
csvpath_trains=['/acmenas/hakrami/3D_lesion_DF/Data/splits/combined_4datasets.csv']
pathBase = '/acmenas/hakrami/patched-Diffusion-Models-UAD/Data_train'
csvpath_val = '/acmenas/hakrami/3D_lesion_DF/splits/IXI_train_fold0.csv'
csvpath_test = '/acmenas/hakrami/3D_lesion_DF/splits/Brats21_sub_test.csv'
var_csv = {}
states = ['train','val','test']

df_list = []

# Loop through each CSV file path and read it into a DataFrame
for csvpath in csvpath_trains:
    df = pd.read_csv(csvpath)
    df_list.append(df)
# %%


var_csv['train'] =pd.concat(df_list, ignore_index=True)
var_csv['val'] = pd.read_csv(csvpath_val)
var_csv['test'] = pd.read_csv(csvpath_test)
# if cfg.mode == 't2':
#     keep_t2 = pd.read_csv(cfg.path.IXI.keep_t2) # only keep t2 images that have a t1 counterpart

for state in states:
    var_csv[state]['settype'] = state
    var_csv[state]['norm_path'] = ''
    var_csv[state]['img_path'] = pathBase  + var_csv[state]['img_path']
    var_csv[state]['mask_path'] = pathBase  + var_csv[state]['mask_path']
    if state != 'test':
        var_csv[state]['seg_path'] = None
    else:
        var_csv[state]['seg_path'] = pathBase  + var_csv[state]['seg_path']

    # if cfg.mode == 't2': 
    #     var_csv[state] =var_csv[state][var_csv[state].img_name.isin(keep_t2['0'].str.replace('t2','t1'))]
    #     var_csv[state]['img_path'] = var_csv[state]['img_path'].str.replace('t1','t2')
    
    
data_train = Train(var_csv['train'],config) 
data_val = Train(var_csv['val'],config)                
data_test = Eval(var_csv['test'],config)



#data_train = Train(pd.read_csv('/project/ajoshi_27/akrami/monai3D/GenerativeModels/data/split/IXI_train_fold0.csv', converters={'img_path': pd.eval}), config)
train_loader = DataLoader(data_train, batch_size=config.get('batch_size', 1),shuffle=True,num_workers=8)

#data_val = Train(pd.read_csv('/project/ajoshi_27/akrami/monai3D/GenerativeModels/data/split/IXI_val_fold0.csv', converters={'img_path': pd.eval}), config)
val_loader = DataLoader(data_val, batch_size=config.get('batch_size', 1),shuffle=True,num_workers=8)

#data_test = Train(pd.read_csv('/project/ajoshi_27/akrami/monai3D/GenerativeModels/data/split/Brats21_test.csv', converters={'img_path': pd.eval}), config)
test_loader = DataLoader(data_test, batch_size=config.get('batch_size', 1),shuffle=False,num_workers=16)



In [ ]:
device = torch.device("cuda")

model = DiffusionModelUNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    num_channels=[256, 256, 512],
    attention_levels=[False, False, True],
    num_head_channels=[0, 0, 512],
    num_res_blocks=2,
)
model_filename = '/acmenas/hakrami/3D_lesion_DF/models/norm3/model_large_epoch349.pt'



model.to(device)
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model.load_state_dict(torch.load(model_filename)) 
scheduler = DDPMScheduler(num_train_timesteps=1000, schedule="scaled_linear_beta", beta_start=0.0005, beta_end=0.0195)

inferer = DiffusionInferer(scheduler)

optimizer = torch.optim.Adam(params=model.parameters(), lr=5e-5)

# Generate an Image

In [ ]:
sub_test = next(iter(test_loader))
print(sub_test.keys())
print(sub_test['age'])

In [ ]:

# Expand the dimensions of sub_test['peak'] to make it [1, 1, 1, 1, 4]
peak_expanded = (sub_test['peak'].unsqueeze(1).unsqueeze(2).unsqueeze(3).unsqueeze(4)).long()
# Move both tensors to the device
image_array = sub_test['vol']['data'].to(device)
seg = sub_test['seg']['data'].to(device)
peak_expanded = peak_expanded.to(device)
image_array = (image_array / peak_expanded)
middle_slice_idx = image_array.size(-1) // 2

In [ ]:
plt.imshow(image_array[0,0][:,:,middle_slice_idx].squeeze().cpu().numpy(), cmap='gray', vmin=0, vmax=2)
plt.title('Outlier Image')
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(seg[0,0][:,:,middle_slice_idx].squeeze().cpu().numpy(), cmap='gray', vmin=0, vmax=2)
plt.title('Segmentation Image')
plt.colorbar()
plt.show()

# Denoise test samples

In [ ]:
def denoise(noised_img,sample_time,scheduler,inferer,model):
    with torch.no_grad():
        with autocast(enabled=True):
            for t in range(sample_time - 1, -1, -1):
                batch_size = noised_img.size(0)  # Get the batch size
                t_batch=torch.Tensor((t,)).to(noised_img.device)
                t_batch = t_batch.unsqueeze(0).expand(batch_size, -1)  # Expand tensor `t` to have the desired batch size
                t_batch = t_batch.to(noised_img.device)[:,0] 
                model_output = model(noised_img, timesteps=t_batch)
                noised_img, _ = scheduler.step(model_output, t, noised_img)
            return noised_img

    

In [ ]:
def denoise_epistemic(noised_img,sample_time,scheduler,inferer,model):
    max_sample_time= sample_time[-1]
    current_time = sample_time[-2]
    i=-2
    noised_img_org = noised_img.clone()
    with torch.no_grad():
        with autocast(enabled=True):
            for t in range(max_sample_time - 1, -1, -1):
                batch_size = noised_img.size(0)  # Get the batch size
                t_batch=torch.Tensor((t,)).to(noised_img.device)
                t_batch = t_batch.unsqueeze(0).expand(batch_size, -1)  # Expand tensor `t` to have the desired batch size
                t_batch = t_batch.to(noised_img.device)[:,0] 
                model_output = model(noised_img, timesteps=t_batch)
                noised_img, _ = scheduler.step(model_output, t, noised_img)
                if t == current_time:
                    noised_img [i,:,:,:,:]= noised_img_org[i,:,:,:,:]
                    i-=1
                    if i != -6: #TODO hard coded
                        current_time = sample_time[i]
            return noised_img

    

In [ ]:
n = 5  # For example, if you want to repeat it 5 times

# Repeat the tensor
repeated_image_array = image_array.repeat(n, 1, 1, 1, 1) 

In [ ]:
i = 0
sample_time = [100,400,600,800,999]
noise =torch.randn_like(repeated_image_array)
noisy_img = scheduler.add_noise(original_samples=repeated_image_array, noise=noise, timesteps=torch.tensor(sample_time))
noised_img = noisy_img.to(device)
middle_slice_idx = repeated_image_array.size(-1) // 2
denoised_sample = denoise_epistemic(noisy_img,sample_time,scheduler,inferer,model)

In [ ]:
i=0
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

# Original Image
axes[0, 0].imshow(repeated_image_array[i][0][:,:,middle_slice_idx].squeeze().cpu().numpy(), vmin=0, vmax=2, cmap='gray')
axes[0, 0].set_title('Original Image')
#recon_img = denoise(image_array.to(device),sample_time,scheduler,inferer,model)

# Original Image
axes[0, 1].imshow(noised_img[i][0][:,:,middle_slice_idx].squeeze().cpu().numpy(), vmin=0, vmax=2, cmap='gray')
axes[0, 1].set_title('noised Image')
#recon_img = denoise(image_array.to(device),sample_time,scheduler,inferer,model)


# Calculate mean and variance along dimension zero
mean_denoised = torch.mean(denoised_sample[:,0][:,:,:].squeeze(), dim=0).cpu().numpy()
variance_denoised = torch.std(denoised_sample[:,0][:,:,:].squeeze(), dim=0).cpu().numpy()
# Original Image
axes[1, 0].imshow(mean_denoised[:,:,middle_slice_idx].squeeze(), vmin=0, vmax=2, cmap='gray')
axes[1, 0].set_title('mean denoised Image')


# Original Image
axes[1, 1].imshow(variance_denoised[:,:,middle_slice_idx].squeeze(), cmap='gray')
axes[1, 1].set_title('std denoised Image')

In [ ]:
i = 0
sample_time = 999
noise =torch.randn_like(repeated_image_array)
noisy_img = scheduler.add_noise(original_samples=repeated_image_array, noise=noise, timesteps=torch.tensor(sample_time))
noised_img = noisy_img.to(device)
middle_slice_idx = repeated_image_array.size(-1) // 2

denoised_sample = denoise(noisy_img,sample_time,scheduler,inferer,model)
    




In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
# Original Image
axes[0, 0].imshow(repeated_image_array[i][0][:,:,middle_slice_idx].squeeze().cpu().numpy(), vmin=0, vmax=2, cmap='gray')
axes[0, 0].set_title('Original Image')
#recon_img = denoise(image_array.to(device),sample_time,scheduler,inferer,model)

# Calculate mean and variance along dimension zero
mean_denoised = torch.mean(denoised_sample[:,0][:,:,:].squeeze(), dim=0).cpu().numpy()
variance_denoised = torch.std(denoised_sample[:,0][:,:,:].squeeze(), dim=0).cpu().numpy()

error = np.abs(repeated_image_array[i][0].squeeze().cpu().numpy()-mean_denoised)

# Original Image
axes[0, 1].imshow(error[:,:,middle_slice_idx].squeeze(), vmin=0, vmax=2, cmap='gray')
axes[0, 1].set_title('noised Image')
#recon_img = denoise(image_array.to(device),sample_time,scheduler,inferer,model)

# Original Image
axes[1, 0].imshow(mean_denoised[:,:,middle_slice_idx].squeeze(), vmin=0, vmax=2, cmap='gray')
axes[1, 0].set_title('mean denoised Image')


# Original Image
axes[1, 1].imshow(variance_denoised[:,:,middle_slice_idx].squeeze(), cmap='gray')
axes[1, 1].set_title('std denoised Image')

In [ ]:

import torch
from scipy.stats import norm
import numpy as np
import statsmodels.stats.multitest as mt

# Sample 3D tensor
x = torch.randn((5, 5, 5))

# Compute z-scores
x =repeated_image_array[0,0,:,:,:].squeeze().cpu().numpy() 
z_scores = (error)/ (variance_denoised+0.000001)

p_values = 2 * (1 - norm.cdf(np.abs(z_scores)))

# # FDR Correction (Benjamini-Hochberg)
# def fdr_correction(p_values, alpha=0.05):
#     m = p_values.size
#     sorted_indices = np.argsort(p_values)
#     sorted_p_values = p_values[sorted_indices]
#     sorted_q_values = np.zeros(m)
#     prev_bh_value = 0

#     for i, p in enumerate(sorted_p_values):
#         bh_value = (p * m) / (i + 1)
#         sorted_q_values[i] = min(bh_value, prev_bh_value)
#         prev_bh_value = sorted_q_values[i]

#     q_values = np.zeros(m)
#     q_values[sorted_indices] = sorted_q_values

#     return q_values

# q_values = fdr_correction(p_values.flatten())

# # Reshape q-values back to the original tensor shape
# q_values_tensor = torch.tensor(q_values.reshape(x.shape))


p_values_flattened = p_values.flatten()
reject, q_values_corrected, _, _ = mt.multipletests(p_values_flattened, method='fdr_bh')
q_values_tensor_corrected = torch.tensor(q_values_corrected.reshape(x.shape))

fig, axes = plt.subplots(2,2, figsize=(10, 10))
axes[0, 0].imshow(x[:,:,middle_slice_idx].squeeze(),cmap='gray')
axes[0, 0].set_title('input')

axes[0, 1].imshow(z_scores[:,:,middle_slice_idx].squeeze(),cmap='gray')
axes[0, 1].set_title('z-score')
#recon_img = denoise(image_array.to(device),sample_time,scheduler,inferer,model)
# Original Image
axes[1, 0].imshow(p_values[:,:,middle_slice_idx].squeeze(), cmap='gray')
axes[1, 0].set_title('p-values')


axes[1, 1].imshow(q_values_tensor_corrected[:,:,middle_slice_idx].squeeze(), cmap='gray')
axes[1, 1].set_title('q-values')




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming p_values is a 3D numpy array containing p-values
alpha = 0.01
outlier_image = np.where(q_values_tensor_corrected< alpha, 1, 0)

# Plotting the middle slice as before
middle_slice_idx = outlier_image.shape[2] // 2
plt.imshow(outlier_image[:,:,middle_slice_idx], cmap='gray')
plt.title('Outlier Image')
plt.colorbar()
plt.show()


In [ ]:
sample_time = 500
noisy_img_500 = scheduler.add_noise(original_samples=repeated_image_array, noise=noise, timesteps=torch.tensor(sample_time))
noised_img_500 = noisy_img_500.to(device)
middle_slice_idx = repeated_image_array.size(-1) // 2

denoised_sample_500 = denoise(noisy_img_500,sample_time,scheduler,inferer,model)
    

In [ ]:
threshold = 0.2618
error_single = np.abs(repeated_image_array[i][0].squeeze().cpu().numpy()-denoised_sample_500[i][0][:,:,:].squeeze().cpu().numpy())
binary_mask = (error_single> threshold).astype(np.float32)
plt.imshow(binary_mask[:,:,middle_slice_idx].squeeze(), cmap='gray')
plt.title('Outlier Image single threshold')
plt.colorbar()
plt.show()


: 

In [ ]:
threshold = 0.417
error_single = np.abs(repeated_image_array[i][0].squeeze().cpu().numpy()-denoised_sample[i][0][:,:,:].squeeze().cpu().numpy())
binary_mask = (error_single> threshold).astype(np.float32)
plt.imshow(binary_mask[:,:,middle_slice_idx].squeeze(), cmap='gray')
plt.title('Outlier Image single threshold')
plt.colorbar()
plt.show()

In [ ]:
# from scipy.ndimage import median_filter
# # Convert tensor to numpy
# outlier_image_median = outlier_image.numpy()

# # Apply median filter
# filtered_array = median_filter(outlier_image_median, size=3)

# # Convert back to tensor
# plt.imshow(filtered_array[:,:,middle_slice_idx], cmap='gray')
# plt.title('Outlier Image filtered')
# plt.colorbar()
# plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you've already calculated z_scores as shown in the previous answers:
# z_scores = (abs(x - mean_denoised)) / variance_denoised

plt.hist(q_values_tensor_corrected.flatten(), bins=30, color='blue', alpha=0.7)
plt.xlabel('Z-Score')
plt.ylabel('Frequency')
plt.title('Histogram of Z-Scores')
plt.grid(True)
plt.show()


In [ ]:


import numpy as np
from scipy.ndimage import binary_dilation

def dilate_3d_scipy(binary_mask, kernel_size=3):
    """
    Dilate a 3D binary mask using a cubic structuring element.

    Parameters:
        binary_mask (numpy.ndarray): 3D binary mask to be dilated.
        kernel_size (int): Size of the cubic structuring element. Default is 3.
        
    Returns:
        numpy.ndarray: Dilated 3D binary mask.
    """
    # Define a 3D structuring element
    kernel = np.ones((kernel_size, kernel_size, kernel_size), bool)
    
    # Dilate (expand) the binary mask
    dilated = binary_dilation(binary_mask, structure=kernel)
    
    return dilated




# Assume 'binary_mask_3d' is your 3D binary mask
# binary_mask_3d = ...

# Dilate the mask
dilated_mask = dilate_3d_scipy(outlier_image, kernel_size=3)


In [ ]:
plt.imshow(dilated_mask[:,:,middle_slice_idx], cmap='gray', vmin=0, vmax=1)
plt.title('Outlier Image single threshold')
plt.colorbar()
plt.show()

In [ ]:
import tqdm
mask = torch.from_numpy(dilated_mask.astype(np.float32)).unsqueeze(0).unsqueeze(0).to(device)
image_array_1= repeated_image_array[0:1,:,:,:,:].to(device)
denosied_image_1 = torch.from_numpy(mean_denoised).unsqueeze(0).unsqueeze(0).to(device)
#mask = torch.rand_like(image_array_1).to(device)
print()

val_image_masked = image_array_1.to(device)*(1-mask)+(mask)*denosied_image_1
val_image_inpainted = denosied_image_1.to(device) 
# plot
print(val_image_inpainted.shape)
scheduler.set_timesteps(num_inference_steps=999)

timesteps =50
num_resample_steps = 10
progress_bar = tqdm.tqdm(reversed(range(timesteps)))

with torch.no_grad():
    with autocast(enabled=True):
        for t in progress_bar:
            for u in range(num_resample_steps):
                # get the known portion at t-1
                if t > 0:
                    noise =  torch.randn_like(image_array_1).to(device)
                    timesteps_prev = torch.Tensor((t - 1,)).to(device).long()
                    val_image_inpainted_prev_known = scheduler.add_noise(
                        original_samples=val_image_masked, noise=noise, timesteps=timesteps_prev
                    )
                else:
                    val_image_inpainted_prev_known = val_image_masked

                # perform a denoising step to get the unknown portion at t-1
                if t > 0:
                    timesteps = torch.Tensor((t,)).to(device).long()
                    model_output = model(val_image_inpainted, timesteps=timesteps)
                    val_image_inpainted_prev_unknown, _ = scheduler.step(model_output, t, val_image_inpainted)

                # combine known and unknown using the mask
                val_image_inpainted = torch.where(
                    mask == 0, val_image_inpainted_prev_known, val_image_inpainted_prev_unknown
                )




# num_resample_steps = 4
# with torch.no_grad():
#     with autocast(enabled=True):
#         for t in progress_bar:
#             for u in range(num_resample_steps):
#                 # get the known portion at t-1
#                 if t > 0:
#                     noise =  torch.randn_like(image_array_1).to(device)
#                     timesteps_prev = torch.Tensor((t - 1,)).to(device).long()
#                     val_image_inpainted_prev_known = scheduler.add_noise(
#                         original_samples=val_image_masked_2, noise=noise, timesteps=timesteps_prev
#                     )
#                 else:
#                     val_image_inpainted_prev_known = val_image_masked_2

#                 # perform a denoising step to get the unknown portion at t-1
#                 if t > 0:
#                     timesteps = torch.Tensor((t,)).to(device).long()
#                     model_output = model(val_image_inpainted_2, timesteps=timesteps)
#                     val_image_inpainted_prev_unknown, _ = scheduler.step(model_output, t, val_image_inpainted_2)

#                 # combine known and unknown using the mask
#                 val_image_inpainted_2 = torch.where(
#                     mask_2 == 1, val_image_inpainted_prev_known, val_image_inpainted_prev_unknown
#                 )



In [ ]:

plt.imshow(val_image_masked[0,0][:,:,middle_slice_idx].squeeze().cpu().numpy(), cmap='gray', vmin=0, vmax=2)
plt.title('Outlier Image single threshold')
plt.colorbar()
plt.show()

In [ ]:

plt.imshow(val_image_inpainted[0,0][:,:,middle_slice_idx].squeeze().cpu().numpy(), cmap='gray', vmin=0, vmax=2)
plt.title('Outlier Image single threshold')
plt.colorbar()
plt.show()

In [ ]:

plt.imshow(denoised_sample_500[0,0][:,:,middle_slice_idx].squeeze().cpu().numpy(), cmap='gray', vmin=0, vmax=2)
plt.title('Outlier Image single threshold')
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(denoised_sample[0,0][:,:,middle_slice_idx].squeeze().cpu().numpy(), cmap='gray', vmin=0, vmax=2)
plt.title('Outlier Image single threshold')
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(torch.abs(val_image_inpainted[0,0][:,:,middle_slice_idx]-repeated_image_array[0,0][:,:,middle_slice_idx]).squeeze().cpu().numpy(), cmap='gray')
plt.title('Outlier Image single threshold')
plt.colorbar()
plt.show()

In [ ]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt


# Save numpy array as .nii image
nii_img = nib.Nifti1Image(val_image_inpainted[0,0].cpu().numpy(), affine=np.eye(4))
nib.save(nii_img, 'sample_image_real_inpaint.nii')



# Load the .nii image
loaded_img = nib.load('sample_image_real_inpaint.nii')
loaded_data = loaded_img.get_fdata()

# Plot a slice from the loaded data
slice_idx = loaded_data.shape[2] // 2
plt.imshow(loaded_data[:, :, slice_idx], cmap='gray')
plt.colorbar()
plt.title(f"Slice {slice_idx}")
plt.show()


In [ ]:
# Save numpy array as .nii image
nii_img = nib.Nifti1Image(denoised_sample_500[0,0].cpu().numpy(), affine=np.eye(4))
nib.save(nii_img, 'sample_image_real_500.nii')

In [ ]:
# Save numpy array as .nii image
nii_img = nib.Nifti1Image(denoised_sample[0,0].cpu().numpy(), affine=np.eye(4))
nib.save(nii_img, 'sample_image_real_999.nii')

In [ ]:
healthy_array = sub_test['path_normal']['data'].to(device)
healthy_array = (healthy_array / peak_expanded)
nii_img = nib.Nifti1Image(healthy_array[0,0].cpu().numpy(), affine=np.eye(4))
nib.save(nii_img, 'sample_image_real_healthy.nii')

In [ ]:

nii_img = nib.Nifti1Image(repeated_image_array[0,0].cpu().numpy(), affine=np.eye(4))
nib.save(nii_img, 'sample_image_real_lesion.nii')